By: 10117094	Muhammad Walidin
    10116476	Idham Rahadian
    10116490	Muhammad Fajar Maulana

In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [2]:
raw = pd.read_csv( "data/data/1.0_data_review_clean_without_stop_removal.csv")
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]
slang_words = pd.concat([slang_words, slang, baku_words])

In [3]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))

In [4]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      804 non-null    int64  
 1   review      804 non-null    object 
 2   avg_word    804 non-null    float64
 3   word_count  804 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 25.2+ KB
None


In [5]:
raw['review'][2]

"['barang', 'yang', 'datang', 'bukan', 'baracuda', 'tiga', 'lima', 'tujuh', 'tujuh', '0rpm', 'tapi', 'pipeline', 'hd']"

In [6]:
def remove_numeric(review):
    return re.sub("\d"," ", review)

raw['review'] = raw['review'].apply(remove_numeric)

In [7]:
raw.head()

,rating,review,avg_word,word_count
0,5,"['tidak', 'ada', 'ulasan']",4.666667,3
1,5,"['puas', 'kecil', 'tapi', 'kencang']",5.000000,4
2,1,"['barang', 'yang', 'datang', 'bukan', 'baracud...",5.000000,13
3,5,"['terima kasih', 'barang', 'sudah', 'sampai', ...",6.176471,17
4,5,"['barang', 'bagus', 'pengirman', 'cepat', 'man...",5.666667,6


In [8]:
def csv_string_to_list(csv_string):
    return csv_string[1:-1].split(",")

def string_without_quotes(word_list):
    new  = []
    for word in word_list:
        new.append(word.replace("'",""))
    return new

def delete_suffix_nya(review):
    return re.sub("(?:nya|ny)[$|\s]","",review)

In [9]:
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review']

0                             ['tidak', 'ada', 'ulasan']
1                   ['puas', 'kecil', 'tapi', 'kencang']
2      ['barang', 'yang', 'datang', 'bukan', 'baracud...
3      ['terima kasih', 'barang', 'sudah', 'sampai', ...
4      ['barang', 'bagus', 'pengirman', 'cepat', 'man...
                             ...                        
799    ['barang', 'kondisi', 'oke', 'respon', 'cepat'...
800    ['barang', 'sampai', 'tepat', 'waktu', 'sesuai...
801    ['pelayanan', 'sangat', 'bagus', 'barang', 'se...
802    ['barang', 'sampai', 'lebih', 'cepat', 'dan', ...
803                  ['bonus', 'bohong', 'tidak', 'ada']
Name: review, Length: 804, dtype: object

In [10]:
raw['review'] = raw['review'].apply(csv_string_to_list)
raw['review']

0                           ['tidak',  'ada',  'ulasan']
1                ['puas',  'kecil',  'tapi',  'kencang']
2      ['barang',  'yang',  'datang',  'bukan',  'bar...
3      ['terima kasih',  'barang',  'sudah',  'sampai...
4      ['barang',  'bagus',  'pengirman',  'cepat',  ...
                             ...                        
799    ['barang',  'kondisi',  'oke',  'respon',  'ce...
800    ['barang',  'sampai',  'tepat',  'waktu',  'se...
801    ['pelayanan',  'sangat',  'bagus',  'barang', ...
802    ['barang',  'sampai',  'lebih',  'cepat',  'da...
803               ['bonus',  'bohong',  'tidak',  'ada']
Name: review, Length: 804, dtype: object

In [11]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [12]:
raw['review']

0                           ['tidak',  'ada',  'ulasan']
1                ['puas',  'kecil',  'tapi',  'kencang']
2      ['barang',  'yang',  'datang',  'bukan',  'bar...
3      ['terima kasih',  'barang',  'sudah',  'sampai...
4      ['barang',  'bagus',  'pengirman',  'cepat',  ...
                             ...                        
799    ['barang',  'kondisi',  'oke',  'respon',  'ce...
800    ['barang',  'sampai',  'tepat',  'waktu',  'se...
801    ['pelayanan',  'sangat',  'bagus',  'barang', ...
802    ['barang',  'sampai',  'lebih',  'cepat',  'da...
803               ['bonus',  'bohong',  'tidak',  'ada']
Name: review, Length: 804, dtype: object

In [13]:
def remove_single_alphabet_only(review):
    return [word for word in review if word not in string.ascii_lowercase]

def remove_too_short_words(review):
    return [word for word in review if len(word) > 2]

In [14]:
raw['review'] = raw['review'].apply(remove_single_alphabet_only)
raw['review'] = raw['review'].apply(remove_too_short_words)

In [15]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
raw['word_count'] = raw['review'].map(len)
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [16]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 803
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      804 non-null    int64  
 1   review      804 non-null    object 
 2   avg_word    804 non-null    float64
 3   word_count  804 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 31.4+ KB


In [17]:
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x.split(",")))

In [18]:
raw.to_csv("data/data/1.1_data_review_clean.csv", index=False)

In [19]:
most_common = pd.Series(''.join(raw['review']).split(",")).value_counts()[:10]
most_common

 'dan'       253
 'tidak'     214
 'yang'      199
 'dengan'    185
 'barang'    180
 'di'        152
 'sesuai'    150
 'saya'      137
 'sudah'     131
 'bagus'     121
dtype: int64

In [20]:
def add_coma(review):
    return review + ','
raw['review'] = raw['review'].apply(add_coma)

In [21]:
least_common = Counter("".join(raw["review"]).split(",")).most_common()[-10:]
least_common

[(" 'help'", 1),
 ("'Cuma'", 1),
 (" 'target'", 1),
 (" 'spekyg'", 1),
 (" 'kosongan'", 1),
 (" 'stil'", 1),
 (" 'great'", 1),
 (" 'sejauh'", 1),
 ("'bonus'", 1),
 ('', 1)]